In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppress TensorFlow logs

In [11]:
# ───────────────────────────────────────────────────────────────
# Install -- run once per notebook
# ───────────────────────────────────────────────────────────────
!pip install --quiet \
    "transformers>=4.41.0" peft unsloth accelerate \
    datasets evaluate bert-score sacrebleu nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 53.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━

In [12]:
# # ─────────────────────────────────────────────────────────────────────────────
# #  Alpaca evaluation  –  ROUGE | BLEU | Precision | BERTScore
# # ─────────────────────────────────────────────────────────────────────────────
# import os, random, torch, nltk, re, pandas as pd
# from datasets import load_dataset
# from evaluate  import load as load_metric
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# # ================ Secrets (optional) ========================================
# HF_TOKEN = None
# try:
#     from kaggle_secrets import UserSecretsClient            # noqa: E402
#     HF_TOKEN = UserSecretsClient().get_secret("HF_TOKEN")
#     print("✅  Hugging Face token retrieved.")
# except Exception:
#     print("ℹ️  No HF token found – assuming the models are public.")

# # ================ Config ====================================================
# BASE_MODEL_ID    = "unsloth/mistral-7b-v0.3-bnb-4bit"
# ADAPTER_MODEL_ID = "ParamDev/mistral-7b-v0.3_alpaca"   # your LoRA

# EVAL_FRACTION      = 0.10          # 10 % of Alpaca for evaluation
# SEED               = 42
# MAX_SAMPLES        = 1000          # cap for speed; set None for full
# MAX_NEW_TOKENS     = 1024
# TEMPERATURE        = 0.3
# TOP_P              = 0.9
# DO_SAMPLE          = True

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"🚀  Using device: {DEVICE}")

# # ================ NLTK (for metrics) =======================================
# def ensure_nltk():
#     for res in ("punkt", "wordnet", "omw-1.4"):
#         try:
#             nltk.data.find(
#                 f"tokenizers/{res}" if res == "punkt" else f"corpora/{res}"
#             )
#         except LookupError:
#             nltk.download(res, quiet=True)
# ensure_nltk()

# # ================ Load Alpaca ==============================================
# def load_alpaca(eval_frac, cap, seed):
#     ds = load_dataset("tatsu-lab/alpaca", split="train")
#     total = len(ds)
#     eval_size = int(total * eval_frac)
#     random.seed(seed)
#     idxs = random.sample(range(total), eval_size)
#     if cap is not None:
#         idxs = idxs[:cap]
#     subset = ds.select(idxs)
#     print(f"🔎  Alpaca evaluation subset: {len(subset)} / {total}")
#     return subset

# # ================ Model & Tokenizer ========================================
# def load_model(base_id, adapter_id, token):
#     try:
#         tok = AutoTokenizer.from_pretrained(adapter_id, token=token)
#     except OSError:
#         tok = AutoTokenizer.from_pretrained(base_id, token=token)
#     tok.pad_token = tok.eos_token

#     base = AutoModelForCausalLM.from_pretrained(
#         base_id,
#         torch_dtype = torch.bfloat16 if DEVICE == "cuda" else torch.float32,
#         device_map  = "auto"         if DEVICE == "cuda" else None,
#         token       = token,
#     )

#     model = PeftModel.from_pretrained(
#         base, adapter_id, token=token, device_map="auto" if DEVICE == "cuda" else None
#     )
#     model.eval()
#     return model, tok

# # ================ Generation ===============================================
# def generate(prompt, model, tok):
#     chat = [{"role": "user", "content": prompt}]
#     ids  = tok.apply_chat_template(chat, tokenize=True,
#                                    add_generation_prompt=True,
#                                    return_tensors="pt").to(model.device)
#     out  = model.generate(
#         ids,
#         max_new_tokens = MAX_NEW_TOKENS,
#         temperature    = TEMPERATURE,
#         top_p          = TOP_P,
#         do_sample      = DO_SAMPLE,
#         pad_token_id   = tok.pad_token_id,
#     )[0][ids.shape[-1]:]
#     return tok.decode(out, skip_special_tokens=True).strip()

# # ================ Simple token-level precision =============================
# tokeniser = re.compile(r"\w+")

# def word_precision(pred, ref):
#     pred_toks = tokeniser.findall(pred.lower())
#     ref_toks  = tokeniser.findall(ref.lower())
#     if not pred_toks:  # avoid div-by-zero
#         return 0.0
#     overlap = sum(1 for t in pred_toks if t in ref_toks)
#     return overlap / len(pred_toks)

# # ================ Main ======================================================
# def main():
#     ds = load_alpaca(EVAL_FRACTION, MAX_SAMPLES, SEED)

#     model, tok = load_model(BASE_MODEL_ID, ADAPTER_MODEL_ID, HF_TOKEN)

#     rouge  = load_metric("rouge")
#     bleu   = load_metric("sacrebleu")
#     bert   = load_metric("bertscore")

#     preds, refs, refs_bleu, precisions = [], [], [], []

#     for i, ex in enumerate(ds, 1):
#         prompt    = ex["instruction"]
#         reference = ex["output"]
#         pred      = generate(prompt, model, tok)

#         preds.append(pred)
#         refs.append(reference)
#         refs_bleu.append([reference])
#         precisions.append(word_precision(pred, reference))

#         if i <= 3:  # show a few examples
#             print(f"\n—— Example {i} ————————————————")
#             print("Prompt    :", prompt[:120].replace("\n", " "))
#             print("Reference :", reference[:120].replace("\n", " "))
#             print("Generated :", pred[:120].replace("\n", " "))

#     # ─── ROUGE ──────────────────────────────────────────────────────────────
#     r = rouge.compute(predictions=preds, references=refs)
#     print("\n📊  ROUGE")
#     for k, v in r.items():
#         print(f"  {k:7} : {v*100:5.2f}")

#     # ─── BLEU (sacrebleu) ───────────────────────────────────────────────────
#     b = bleu.compute(predictions=preds, references=refs_bleu)
#     print("\n📊  BLEU")
#     print(f"  Score   : {b['score']:.2f}")

#     # ─── Simple token precision ────────────────────────────────────────────
#     tp = sum(precisions) / len(precisions)
#     print("\n📊  Token Precision (word overlap)")
#     print(f"  Precision : {tp*100:.2f}")

#     # ─── BERTScore ──────────────────────────────────────────────────────────
#     bs = bert.compute(predictions=preds, references=refs,
#                       lang="en", rescale_with_baseline=True)
#     P = sum(bs["precision"]) / len(bs["precision"])
#     R = sum(bs["recall"])    / len(bs["recall"])
#     F = sum(bs["f1"])        / len(bs["f1"])
#     print("\n📊  BERTScore (rescaled)")
#     print(f"  Precision : {P*100:.2f}")
#     print(f"  Recall    : {R*100:.2f}")
#     print(f"  F1        : {F*100:.2f}")

#     print("\n✅  Evaluation complete.")

# if __name__ == "__main__":
#     main()


In [13]:
# ───────────────────────────────────────────────────────────────
#  Evaluate UnsLoTH-LoRA model on yahma/alpaca-cleaned
#  Metrics: ROUGE | BLEU | word-precision | BERTScore
# ───────────────────────────────────────────────────────────────
import os, random, re, torch, nltk
from datasets import load_dataset
from evaluate import load as load_metric
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ----------------------------
# Secrets (optional, for HF Hub)
# ----------------------------
HF_TOKEN = None
try:
    from kaggle_secrets import UserSecretsClient            # noqa: E402
    HF_TOKEN = UserSecretsClient().get_secret("HF_TOKEN")
    print("✅  Hugging Face token retrieved.")
except Exception:
    print("ℹ️  No HF token found – assuming public models.")

# ----------------------------
# Config
# ----------------------------
BASE_MODEL_ID    = "unsloth/mistral-7b-v0.3-bnb-4bit"
ADAPTER_MODEL_ID = "ParamDev/mistral-7b-v0.3_alpaca"     # your LoRA adapter

EVAL_FRACTION   = 0.10    # 10 % random slice of Alpaca-cleaned
MAX_SAMPLES     = 1000    # cap for speed.  Set None to use the full slice
SEED            = 42
MAX_NEW_TOKENS  = 256
TEMPERATURE     = 0.3
TOP_P           = 0.9
DO_SAMPLE       = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀  Using device: {DEVICE}")

# ----------------------------
# NLTK resources (ROUGE / BLEU helper)
# ----------------------------
for res in ("punkt", "wordnet", "omw-1.4"):
    try:
        path = ("tokenizers/" if res == "punkt" else "corpora/") + res
        nltk.data.find(path)
    except LookupError:
        nltk.download(res, quiet=True)

# ----------------------------
# Data
# ----------------------------
def load_alpaca_cleaned(frac, cap, seed):
    ds = load_dataset("yahma/alpaca-cleaned", split="train")
    total = len(ds)
    eval_size = int(total * frac)
    random.seed(seed)
    idxs = random.sample(range(total), eval_size)
    if cap is not None:
        idxs = idxs[:cap]
    sub = ds.select(idxs)
    print(f"🔎  Evaluation subset: {len(sub)} / {total}")
    return sub

# ----------------------------
# Model & tokenizer
# ----------------------------
def load_model(base_id, adapter_id, token):
    try:
        tok = AutoTokenizer.from_pretrained(adapter_id, token=token)
    except OSError:
        tok = AutoTokenizer.from_pretrained(base_id, token=token)
    tok.pad_token = tok.eos_token

    base = AutoModelForCausalLM.from_pretrained(
        base_id,
        torch_dtype = torch.bfloat16 if DEVICE == "cuda" else torch.float32,
        device_map  = "auto" if DEVICE == "cuda" else None,
        token       = token,
    )

    model = PeftModel.from_pretrained(
        base, adapter_id, token=token, device_map="auto" if DEVICE == "cuda" else None
    )
    model.eval()
    return model, tok

# ----------------------------
# Generation
# ----------------------------
def generate_response(prompt, model, tok):
    chat = [{"role": "user", "content": prompt}]
    ids  = tok.apply_chat_template(chat, tokenize=True,
                                   add_generation_prompt=True,
                                   return_tensors="pt").to(model.device)
    out  = model.generate(
        ids,
        max_new_tokens = MAX_NEW_TOKENS,
        temperature    = TEMPERATURE,
        top_p          = TOP_P,
        do_sample      = DO_SAMPLE,
        pad_token_id   = tok.pad_token_id,
    )[0][ids.shape[-1]:]
    return tok.decode(out, skip_special_tokens=True).strip()

# ----------------------------
# Lexical precision (word-overlap)
# ----------------------------
_tokeniser = re.compile(r"\w+")
def word_precision(pred, ref):
    pt = _tokeniser.findall(pred.lower())
    rt = _tokeniser.findall(ref.lower())
    return 0.0 if not pt else sum(1 for t in pt if t in rt) / len(pt)

# ----------------------------
# Main
# ----------------------------
def main():
    ds = load_alpaca_cleaned(EVAL_FRACTION, MAX_SAMPLES, SEED)
    model, tok = load_model(BASE_MODEL_ID, ADAPTER_MODEL_ID, HF_TOKEN)

    rouge = load_metric("rouge")
    bleu  = load_metric("sacrebleu")
    bert  = load_metric("bertscore")

    preds, refs, refs_bleu, precs = [], [], [], []

    for i, ex in enumerate(ds, 1):
        # Re-create the same *semantic* prompt the model was trained on
        # (notice we do NOT append the target/output)
        if ex["input"]:
            prompt = f"{ex['instruction']}\n{ex['input']}"
        else:
            prompt = ex["instruction"]

        reference = ex["output"]
        pred      = generate_response(prompt, model, tok)

        preds.append(pred)
        refs.append(reference)
        refs_bleu.append([reference])
        precs.append(word_precision(pred, reference))

        if i <= 3:   # quick peek
            print(f"\n—— Example {i} ————————————————")
            print("Prompt    :", prompt[:120].replace('\n', ' '))
            print("Reference :", reference[:120].replace('\n', ' '))
            print("Generated :", pred[:120].replace('\n', ' '))

    # ---------- metrics ----------
    r = rouge.compute(predictions=preds, references=refs)
    print("\n📊  ROUGE")
    for k, v in r.items():
        print(f"  {k:7} : {v*100:5.2f}")

    b = bleu.compute(predictions=preds, references=refs_bleu)
    print("\n📊  BLEU (sacrebleu)")
    print(f"  Score   : {b['score']:.2f}")

    tp = sum(precs) / len(precs)
    print("\n📊  Word Precision")
    print(f"  Precision : {tp*100:.2f}")

    bs = bert.compute(predictions=preds, references=refs,
                      lang="en", rescale_with_baseline=True)
    P = sum(bs["precision"]) / len(bs["precision"])
    R = sum(bs["recall"])    / len(bs["recall"])
    F = sum(bs["f1"])        / len(bs["f1"])
    print("\n📊  BERTScore (rescaled)")
    print(f"  Precision : {P*100:.2f}")
    print(f"  Recall    : {R*100:.2f}")
    print(f"  F1        : {F*100:.2f}")

    print("\n✅  Evaluation complete.")

if __name__ == "__main__":
    main()


RuntimeError: operator torchvision::nms does not exist